# Build JSON table of planet properties

for use by `batman` package (and others) with data from the NASA Exoplanet Archive and the Exoplanet Orbit Database.

In [78]:
%matplotlib inline
from astroquery.exoplanet_orbit_database import ExoplanetOrbitDatabase
from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive
from astropy.constants import G
import numpy as np

stars = ['Kepler-62', 'Kepler-296', 'TRAPPIST-1']
teffs = [4926, 3526, 2559]
limb_darkening = [[0.1082, 0.1483], [0.125, 0.337], [0.10, 0.57]]
ld_ref = ['spitzer3.5_claret2013', 'H_claret1995', 'I_gillon2016']
planet_lists = [list('bcdef'), list('bcdef'), list('bcdefgh')]

In [81]:
# t0, per, rp, a, inc, ecc, w, limb_dark, u

def keplers_third(row): 
    """
    Given an orbital period and stellar mass, return semimajor axis
    in units of stellar radii
    """
    a = (row['pl_orbper']**2 * G*row['st_mass']/4/np.pi**2)**(1/3)
    return (a/row['st_rad']).decompose().value

planet_properties = dict()

for star, planet_list, ld in zip(stars, planet_lists, limb_darkening):
    planet_dict = dict()
    for planet in planet_list: 
        planet_identifier = "{0} {1}".format(star, planet)
        try: 
            eod = ExoplanetOrbitDatabase.query_planet(planet_identifier)
            t0 = eod['TT']
        except KeyError: 
            t0 = 0

        nea = NasaExoplanetArchive.query_planet(planet_identifier)
    
        planet_radius = nea['pl_radj']
        stellar_radius = nea['st_rad']
        rp = (planet_radius/stellar_radius).decompose().value
        ecc = nea['pl_orbeccen']

        if hasattr(ecc, 'value'):
            ecc = ecc.value
        else: 
            ecc = 0
        
        props = dict(per=nea['pl_orbper'].value, t0=t0, rp=rp, a=keplers_third(nea), 
                     inc=nea['pl_orbincl'].value, ecc=ecc, w=90, 
                     limb_dark='quadratic', u=ld)
        planet_dict[planet] = props
    planet_properties[star] = planet_dict

In [82]:
planet_properties['TRAPPIST-1']['e'] = 2450000 + 7660.37859 # Gillon 2017
planet_properties['TRAPPIST-1']['f'] = 2450000 + 7671.39767 # Gillon 2017
planet_properties['TRAPPIST-1']['g'] = 2450000 + 7665.34937 # Gillon 2017
planet_properties['TRAPPIST-1']['h'] = 2457662.55284 # Luger 2017

In [83]:
import json
import os

systems_path = '../libra/data/systems.json'

overwrite = False
if not os.path.exists(systems_path) or overwrite:
    json.dump(planet_properties, open(systems_path, 'w'), 
              indent=4)